In [2]:
#!pip install kagglehub
#!pip install pandas
#!pip install nltk
#!pip install transformers
#!pip install torch torchvision torchaudio
#!pip install emoji

In [3]:
# Imports Dataset Kaggle
import kagglehub
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from transformers import BertTokenizer
import emoji
# Download latest version
path = kagglehub.dataset_download("mariumfaheem666/spam-sms-classification-using-nlp")+"/"+"Spam_SMS.csv"
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_eng')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

[nltk_data] Downloading package punkt to /home/aletyska/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/aletyska/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [91]:
# Definição de Classes
class Mensagem:
    def __init__(self, mensagem_original, classificacao):
        self.MensagemOriginal = mensagem_original
        self.Classificacao = classificacao

        #variável temporária para tratamentos
        text = self.MensagemOriginal
        text = emoji.demojize(text, language='en')
        text = re.sub(r'https?://\S+|www\.\S+', '', text)
        text = text.encode("utf-8", "ignore").decode()  
        text = re.sub(r'<.*?>', '', text)
        text = re.sub(r'\s+', ' ', text)
        sentences = sent_tokenize(text)
        cleaned_sentences = [s.strip() for s in sentences]
        
        # Msg Tratada (limpa)
        self.MensagemTratada = ' '.join(cleaned_sentences)
        
        # Msg Tokenizada
        self.MensagemTokenizada = word_tokenize(self.MensagemTratada)

        # Msg Tokenizada para o Bert
        self.MensagemTokenizadaBert = tokenizer.encode_plus(
            self.MensagemTratada,
            add_special_tokens=True,     # Add [CLS] and [SEP]
            padding='max_length',        # Pad to max length (or truncate)
            truncation=True,             # Truncate to fit max length
            max_length=128,              # You can adjust this
            return_tensors='pt'          # Return PyTorch tensors
        )
        
        self.QuantidadeCaracteres = len(self.MensagemTratada)
        self.QuantidadePalavras = len(self.MensagemTokenizada)

        tagged = pos_tag(self.MensagemTokenizada)
        verbs = [word for word, tag in tagged if tag.startswith('VB')]
        self.QuantidadeVerbos = len(verbs)
        nouns = [word for word, tag in tagged if tag.startswith('NN')]
        self.QuantidadeSubstantivos = len(nouns)

    def ConstroiArvoreSintagmas(self):
        # Lógica para construir a árvore de sintagmas
        self.ArvoreSintagmas = None

    def ExibirDadosMensagem(self):
        print('Mensagem Original: '+ self.MensagemOriginal)
        print('Classificação: '+ self.Classificacao)
        print('-----------------------------------------------------------------------------------------------------')
        print('Mensagem Tratada: '+ self.MensagemTratada)
        print('Mensagem Tokenizada: '+ '|'.join(self.MensagemTokenizada))
        print('Quantidade de Caracteres: '+ str(self.QuantidadeCaracteres))
        print('Quantidade de Palavras: '+ str(self.QuantidadePalavras))
        print('Quantidade de Verbos: '+ str(self.QuantidadeVerbos))
        print('Quantidade de Substantivos: '+ str(self.QuantidadeSubstantivos))
        


class BaseMensagens:
    def __init__(self, listamsgs):
        self.BaseMensagens = []
        for index, row in listamsgs.iterrows():
            self.BaseMensagens.append(Mensagem(row['Message'],row['Class']))

    def Exec(self, percent):
        from collections import defaultdict
        import random
        grouped = defaultdict(list)
        
        for msg in base.BaseMensagens:
            grouped[msg.Classificacao].append(msg)
        
        print('\n###### Original Dataset ######')
        for classificacao, group in grouped.items():
            random.shuffle(grouped[classificacao])
            print(f"{classificacao}: {len(group)} messages")
        
        train = []
        test = []
        for group_items in grouped.values():
            k = max(1, int(len(group_items) * percent / 100))
            train.extend(group_items[:k])  # or use random.sample() if you want it shuffled
            test.extend(group_items[k:])

        grouped = defaultdict(list)
        for msg in train:
            grouped[msg.Classificacao].append(msg)
        print('\n###### Train Dataset ######')
        for classificacao, group in grouped.items():
            print(f"{classificacao}: {len(group)} messages")

        grouped = defaultdict(list)
        for msg in test:
            grouped[msg.Classificacao].append(msg)
        print('\n###### Test Dataset ######')
        for classificacao, group in grouped.items():
            print(f"{classificacao}: {len(group)} messages")

        print('\n###### Returning Datasets ######')
        return BaseTreinamento(train), BaseTeste(test)   

    def MostrarBalanceamento(self):
        from collections import defaultdict
        grouped = defaultdict(list)
        
        for msg in base.BaseMensagens:
            grouped[msg.Classificacao].append(msg)

        print('\n###### Original Dataset ######')
        for classificacao, group in grouped.items():
            print(f"{classificacao}: {len(group)} messages")    
            

class BaseTreinamento(BaseMensagens):
    def __init__(self, base):
        self.BaseMensagens = base

    def Exec(self):
        # Lógica específica de treinamento
        pass

class BaseTeste(BaseMensagens):
    def __init__(self, base):
        self.BaseMensagens = base
        self.Acuracidade = 0.0

    def Exec(self):
        self.predict_model()
        self.calcular_acuracidade()

    def predict_model(self):
        pass

    def calcular_acuracidade(self):
        pass

    def calcular_acuracidade(self):
        total_mensagens = len(self.BaseMensagens)
        acertos = 0

        for mensagem in self.BaseMensagens:
            if mensagem.Tipo == self.predict_model(mensagem):
                acertos += 1

        self.Acuracidade = acertos / total_mensagens


In [82]:
spam_sms = pd.read_csv(path)

In [83]:
base = BaseMensagens(spam_sms)

In [84]:
base.MostrarBalanceamento()


###### Original Dataset ######
ham: 4827 messages
spam: 747 messages


In [85]:
train, test = base.Exec(percent=70)


###### Original Dataset ######
ham: 4827 messages
spam: 747 messages

###### Train Dataset ######
ham: 3378 messages
spam: 522 messages

###### Test Dataset ######
ham: 1449 messages
spam: 225 messages

###### Returning Datasets ######


In [86]:
train

In [90]:
test